# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import  gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Load dataframe of interest
city_data_df=pd.read_csv("../Weatherpy/cities_data.csv")
city_data_df

,Unnamed: 0,City,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Country
0,0,Cape Town,75,1598756403,93,-33.93,18.42,39.20,2.24,ZA
1,1,Te Anau,100,1598756455,71,-45.42,167.72,51.89,7.18,NZ
2,2,Ushuaia,90,1598756461,91,-54.80,-68.30,32.00,20.80,AR
3,3,Yellowknife,75,1598756443,81,62.46,-114.35,46.40,18.34,CA
4,4,Rikitea,10,1598756680,76,-23.12,-134.97,70.12,7.45,PF
...,...,...,...,...,...,...,...,...,...,...
580,580,Cayenne,93,1598756530,94,4.93,-52.33,78.80,2.24,GF
581,581,Sur,37,1598756780,59,22.57,59.53,86.74,2.06,OM
582,582,Burgeo,100,1598756781,81,47.62,-57.62,55.78,2.86,CA
583,583,Bowen,75,1598756745,50,-20.02,148.23,75.20,20.80,AU


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Long' into  locations 
locations_df = city_data_df[["Lat", "Lng"]].astype(float)

# Convert Humidity to float and store
humidity_df = city_data_df["Humidity"].astype(float)

In [5]:
# Plot Heatmap
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Use the gmaps.figure passing a zoom_level of 2 and a center point so the map displays properly at
# a readable size
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations_df, weights=humidity_df, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)

# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] > 70) & (city_data_df["Max Temp"] < 80) & (city_data_df["Cloudiness"] == 0) & (city_data_df["Wind Speed"] < 10), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,Unnamed: 0,City,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Country
0,112,Pozo Colorado,0,1598756578,33,-23.49,-58.80,78.57,6.51,PY
1,180,Jumla,0,1598756710,54,29.27,82.18,70.61,2.42,NP
2,202,Flinders,0,1598756714,45,-34.58,150.86,73.99,8.01,AU
3,224,Port Macquarie,0,1598756630,53,-31.43,152.92,73.99,1.99,AU
4,228,Fomboni,0,1598756717,70,-12.28,43.74,76.39,6.98,KM


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotellist = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Lat']
    lng = narrowed_city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
        
narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,Unnamed: 0,City,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Country,Hotel Name
0,112,Pozo Colorado,0,1598756578,33,-23.49,-58.80,78.57,6.51,PY,Villa Hayes
1,180,Jumla,0,1598756710,54,29.27,82.18,70.61,2.42,NP,Chandannath
2,202,Flinders,0,1598756714,45,-34.58,150.86,73.99,8.01,AU,Wollongong
3,224,Port Macquarie,0,1598756630,53,-31.43,152.92,73.99,1.99,AU,Port Macquarie
4,228,Fomboni,0,1598756717,70,-12.28,43.74,76.39,6.98,KM,Fomboni


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…